In [4]:
import os
import cv2
import numpy as np
from osgeo import gdal
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [7]:
dataset_path = "/home/admin1/Selvin/BE/EuroSAT_MS/"

In [8]:
subdirs = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

# Initialize lists to store images and labels
images = []
labels = []

# Initialize a label counter
label = 0

# Loop through each subdirectory
for subdir in subdirs:
    # List all image files in the subdirectory
    image_files = [os.path.join(dataset_path, subdir, f) for f in os.listdir(os.path.join(dataset_path, subdir)) if f.endswith(".tif")]
    
    # Load images from the files
    for file in image_files:
        # Open the image file using GDAL
        dataset = gdal.Open(file)

        # Read all image channels into a list
        channels = [dataset.GetRasterBand(i + 1).ReadAsArray() for i in range(dataset.RasterCount)]

        # Stack the channels to create a multi-channel image
        image = np.stack(channels, axis=-1)

        # Resize the image to a fixed size (if needed)
        image = cv2.resize(image, (64, 64))  # Adjust the dimensions as per your requirements
        
        # Append the image and label to the lists
        images.append(image)
        labels.append(label)  # Use the label counter as the label
    
    # Increment the label counter for the next subdirectory
    label += 1

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Normalize pixel values to be between 0 and 1
images = images / 255.0

In [9]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, images.shape[-1])))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())


model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))



model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
0.96  


model.add(Dense(8, activation='relu'))
model.add(BatchNormalization())
# model.add(Dropout(0.5))



model.add(Dense(4, activation='relu'))
model.add(BatchNormalization())


model.add(Dense(10, activation='softmax'))

In [6]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('Classical_best_model.h5', save_best_only=True)

# Train the model with callbacks
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping, model_checkpoint])

class_names = ['SeaLake', 'Industrial', 'HerbaceousVegetation', 'PermanentCrop', 'AnnualCrop', 'Residential', 'Pasture', 'Forest', 'Highway', 'River']


Epoch 1/30
675/675 [==============================] - 28s 40ms/step - loss: 1.4333 - accuracy: 0.5409 - val_loss: 0.9436 - val_accuracy: 0.7061
Epoch 2/30
  3/675 [..............................] - ETA: 23s - loss: 1.1288 - accuracy: 0.6354

/home/admin1/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


675/675 [==============================] - 25s 37ms/step - loss: 0.8636 - accuracy: 0.7466 - val_loss: 1.5715 - val_accuracy: 0.6635
Epoch 3/30
675/675 [==============================] - 25s 38ms/step - loss: 0.6711 - accuracy: 0.8096 - val_loss: 0.8542 - val_accuracy: 0.7170
Epoch 4/30
675/675 [==============================] - 25s 36ms/step - loss: 0.5341 - accuracy: 0.8501 - val_loss: 0.6484 - val_accuracy: 0.7880
Epoch 5/30
675/675 [==============================] - 25s 37ms/step - loss: 0.4478 - accuracy: 0.8744 - val_loss: 0.4275 - val_accuracy: 0.8694
Epoch 6/30
675/675 [==============================] - 25s 37ms/step - loss: 0.4132 - accuracy: 0.8826 - val_loss: 0.6727 - val_accuracy: 0.7841
Epoch 7/30
675/675 [==============================] - 23s 34ms/step - loss: 0.3612 - accuracy: 0.8964 - val_loss: 0.3792 - val_accuracy: 0.8930
Epoch 8/30
675/675 [==============================] - 24s 36ms/step - loss: 0.3231 - accuracy: 0.9104 - val_loss: 0.7965 - val_accuracy: 0.7700
Epo

In [8]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
training_loss, training_accuracy = model.evaluate(X_train, y_train)

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

print()


print("Training loss:", training_loss)
print("Training accuracy:", training_accuracy)

675/675 [==============================] - 8s 12ms/step - loss: 0.2394 - accuracy: 0.9239
Test loss: 0.3251367211341858
Test accuracy: 0.896481454372406

Training loss: 0.23944270610809326
Training accuracy: 0.9238888621330261


In [5]:
loaded_model = load_model('Classical_best_model.h5')

In [6]:
predictions = loaded_model.predict(X_test)

for i in range(len(predictions)):
    predicted_label = np.argmax(predictions[i])
    true_label = y_test[i]
    print(f"Predicted: {predicted_label}, True: {true_label}")

NameError: name 'X_test' is not defined

In [10]:
custom_image_path = '/home/admin1/Selvin/BE/EuroSAT_MS/SeaLake/SeaLake_1063.tif'

dataset = gdal.Open(custom_image_path)

channels = [dataset.GetRasterBand(i+1).ReadAsArray() for i in range(dataset.RasterCount)]

image = np.stack(channels, axis=-1)

image = cv2.resize(image, (64, 64))

image = image/ 255.0

image = np.expand_dims(image, axis=0)

predictions = model.predict(image)

predicted_label = np.argmax(predictions)

predicted_class_name = class_names[predicted_label]

# Print the predicted label
print("Predicted Label:", predicted_class_name)

1/1 [==============================] - 0s 20ms/step
Predicted Label: SeaLake


In [12]:
from sklearn.metrics import classification_report

y_pred = loaded_model.predict(X_test)

# Convert predictions to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Print the classification report
print("Classification Report:")
print(classification_report(y_test, y_pred_labels))

169/169 [==============================] - 3s 16ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       595
           1       0.97      0.95      0.96       499
           2       0.86      0.87      0.86       616
           3       0.74      0.86      0.80       503
           4       0.92      0.79      0.85       598
           5       0.98      0.92      0.95       612
           6       0.89      0.61      0.72       383
           7       0.78      0.99      0.87       582
           8       0.91      0.86      0.88       507
           9       0.90      0.97      0.94       505

    accuracy                           0.89      5400
   macro avg       0.89      0.88      0.88      5400
weighted avg       0.90      0.89      0.89      5400

